In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
from pathlib import Path
import pickle
import operator
sys.setrecursionlimit(10000)

In [2]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.analyze_eval import *

In [3]:
import scipy.signal
import skimage.io as sio

In [4]:
outputdir = '/home/pk/Documents/3DForkPlots/chromosome/'

In [5]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/good_models/model_8865_venus_low_ph.pkl')
with open(model_path, 'rb') as f:
    model = pickle.load(f)

Save training fluorescence images and location of cell masks

In [6]:
savedir = outputdir+'/'+'train_images'
eval_imgs_all = model.evaluation_params['eval_imgs']
for i in range(len(eval_imgs_all)):
    filename = savedir+'/'+f'{i:04d}'+'.tiff'
    sio.imsave(filename,eval_imgs_all[i],plugin='tifffile')

In [7]:
eval_filenames=model.evaluation_params['eval_filenames']
txtfile = outputdir+'/'+ 'train_cell_masks.txt'
with open(txtfile, 'w') as f:
    for i in range(len(eval_filenames)):
         f.write(eval_filenames[i].as_posix()+"\n")
f.close()

Save predicted and true dot localizations on training data

In [8]:
eval2 = eval_imgs_all
plot_num = None
preds_raw, n_per_img, plot_area = recognition(model=model, eval_imgs_all=eval2, batch_size=512, use_tqdm=False,
                                     nms=True, candidate_threshold=0.1,
                                     nms_threshold=0.5,
                                     pixel_nm=model.data_generator.psf_params['pixel_size_xy'],
                                     plot_num=None,
                                     win_size=model.data_generator.simulation_params['train_size'],
                                     padding=True,
                                     padded_background=model.simulation_params['bg_values'])

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]


In [9]:
import csv
csvfile = outputdir+'/'+'decode_train_output.csv'
with open(csvfile, 'w') as f:
    table = csv.writer(f)
    header=['molecule','image','x','y','z','photon','prob',\
            'sigma_x','sigma_y','sigma_z','sigma_photon',\
            'x_offset','y_offset','z_offset']
    table.writerow(header)
    for i in range(len(preds_raw)):
        table.writerow(preds_raw[i])
f.close()

In [10]:
trues_raw=model.evaluation_params['ground_truth']
csvfile = outputdir+'/'+'train_data.csv'
with open(csvfile, 'w') as f:
    table = csv.writer(f)
    header=['molecule','image','x','y','z','photon',]
    table.writerow(header)
    for i in range(len(trues_raw)):
        table.writerow(trues_raw[i])
f.close()

In [11]:
trues_raw=model.evaluation_params['ground_truth']
csvfile = outputdir+'/'+'train_data_dummy_sigmas.csv'
with open(csvfile, 'w') as f:
    table = csv.writer(f)
    header=['molecule','image','x','y','z','photon',\
            'sigma_x', 'sigma_y', 'sigma_z','sigma_photon',\
            'x_offset','y_offset','z_offset']
    table.writerow(header)
    for i in range(len(trues_raw)):
        trues_raw[i].extend([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        table.writerow(trues_raw[i])
f.close()

### Preds list has the following in order

    1. counter of the molecule 
    2. image number used to index into the number of the image in the prediction arrays
    3. x position in nm where 0 is top left corner
    4. y position in nm where 0 is top left corner
    5. z position in nm where 0 is from the reference 0 nm in height
    6. photon counts 
    7. probability afer nms
    8. x_sigma in nm 
    9. y_sigma in nm
    10. z_sigma in nm
    11. photon_counts_sigma
    12. x offset
    13. y offset
